# Mapping for COVID Vaccine Access Needs

## Overview

From [Without a Ride, Many in Need Have No Shot at COVID-19 Vaccine](https://www.pewtrusts.org/en/research-and-analysis/blogs/stateline/2021/02/01/without-a-ride-many-in-need-have-no-shot-at-covid-19-vaccine)

"While state and local governments have been busy planning for and distributing vaccines, many have left out an important piece: how to provide transportation to people who can’t get to those sites.

Millions of older adults and low-income people of color who are at higher risk of contracting the virus don’t have cars, don’t drive or don’t live near public transit. Some are homebound. Some live in rural areas far from vaccination sites."



## Import Libraries

In [2]:
from census import Census
from us import states
import pandas as pd 
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely import wkt
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib notebook

In [3]:
engine = create_engine('postgresql://amy@localhost:5432/amy')
c = Census("be588bd861cbf7e0545af372027fc4980855f07e")

## Data

### Census

#### Household Income

In [ ]:
# Table B19013: Median Household Income
# https://censusreporter.org/tables/B19013/

hh_inc = pd.DataFrame.from_dict(c.acs5.get(('NAME', 'group(B19013)'),
          {'for': 'tract:*',
           'in': 'state:{} county:059'.format(states.GA.fips)}))

hh_inc.head()

In [ ]:
hh_inc.to_sql('cc_hh_inc_acs', con=engine, if_exists='replace')

#### Age

In [ ]:
# Table B01001: Sex by Age
# https://www.socialexplorer.com/data/ACS2015_5yr/metadata/?ds=ACS15_5yr&table=B01001

age = pd.DataFrame.from_dict(c.acs5.get(('NAME', 'group(B01001)'),
          {'for': 'tract:*',
           'in': 'state:{} county:059'.format(states.GA.fips)}))

age.head()

In [ ]:
age.to_sql('cc_age_acs', con=engine)

#### Minority Population

In [ ]:
# Table B02001: Race
# https://www.socialexplorer.com/data/ACS2015/metadata/?ds=ACS15&table=B02001

race = pd.DataFrame.from_dict(c.acs5.get(('NAME', 'group(B02001)'),
          {'for': 'tract:*',
           'in': 'state:{} county:059'.format(states.GA.fips)}))

race.head()

In [ ]:
race.to_sql('cc_race_acs', con=engine)

### List of Inputs

In [ ]:
hh_inc = 'cc_hh_inc_acs' # Median Household Income from the American Community Survey
age = 'cc_age_acs'
race = 'cc_race_acs'
sld = 'cc_sld' # https://www.epa.gov/smartgrowth/smart-location-mapping#SLD

### Compile Dataset

In [4]:
query = """
    DROP TABLE IF EXISTS cc_transpo_needs;
    CREATE TABLE cc_transpo_needs AS (
        WITH hh_inc AS (
            SELECT DISTINCT geoid10
            , "B19013_001E" AS median_hh_inc
            FROM cc_sld sld
            JOIN cc_hh_inc_acs h ON h.tract = sld.trfips
            WHERE 1=1
            AND "B19013_001E" > 0
        )
        , age_tract AS (
            SELECT DISTINCT tract
            , "B01001_001E" as total_pop
            , ("B01001_020E" + "B01001_021E" + "B01001_022E" + "B01001_023E" + "B01001_024E" + "B01001_025E" + "B01001_044E" + "B01001_045E" + "B01001_046E" + "B01001_047E" + "B01001_048E" + "B01001_049E") as above_65
            , (("B01001_020E" + "B01001_021E" + "B01001_022E" + "B01001_023E" + "B01001_024E" + "B01001_025E" + "B01001_044E" + "B01001_045E" + "B01001_046E" + "B01001_047E" + "B01001_048E" + "B01001_049E")::FLOAT/"B01001_001E")*100 as perc_65_above
            from cc_age_acs
            WHERE 1=1
            AND "B01001_001E" > 0
            ORDER BY 1
        )
        , age AS (
            SELECT DISTINCT geoid10
            , perc_65_above 
            FROM cc_sld sld
            JOIN age_tract at ON at.tract = sld.trfips
            WHERE 1=1
        )
        , race_tract AS (
            SELECT DISTINCT tract
            , "B02001_001E" as total_pop
            , ("B02001_001E" - "B02001_002E") as non_white
            , (("B02001_001E" - "B02001_002E")::FLOAT/"B02001_001E")*100 as perc_non_white
            from cc_race_acs
            WHERE 1=1
            AND "B02001_001E" > 0
            ORDER BY 1
        )
        , race AS (
            SELECT DISTINCT geoid10
            , perc_non_white 
            FROM cc_sld sld
            JOIN race_tract rt ON rt.tract = sld.trfips
            WHERE 1=1
        )
        SELECT DISTINCT
        -- General
        sld.geoid10
        , sld.geom
        , sld.ac_land AS land_area
        , sld.d1b AS pop_density
        
        -- Demographic
        , median_hh_inc
        , perc_65_above 
        , perc_non_white
        
        -- Transportation
        , d4a AS dist_to_transit
        , d4d AS transit_freq
        
        FROM cc_sld sld 
        LEFT JOIN hh_inc h ON h.geoid10 = sld.geoid10
        LEFT JOIN age a ON a.geoid10 = sld.geoid10
        LEFT JOIN race r ON r.geoid10 = sld.geoid10
        
        WHERE 1=1
    );
"""

engine.execute(query)

## Index

In [5]:
# Add Percentiles

query = """
    ALTER TABLE cc_transpo_needs ADD COLUMN hh_inc_ntile FLOAT;
    ALTER TABLE cc_transpo_needs ADD COLUMN age_ntile FLOAT;
    ALTER TABLE cc_transpo_needs ADD COLUMN race_ntile FLOAT;
    ALTER TABLE cc_transpo_needs ADD COLUMN density_ntile FLOAT;
    ALTER TABLE cc_transpo_needs ADD COLUMN transit_dist_ntile FLOAT;
    ALTER TABLE cc_transpo_needs ADD COLUMN transit_freq_ntile FLOAT;

    WITH percentiles AS (
        SELECT geoid10
        , PERCENT_RANK() OVER (ORDER BY median_hh_inc DESC) AS hh_inc_ntile
        , PERCENT_RANK() OVER (ORDER BY perc_65_above) AS age_ntile
        , PERCENT_RANK() OVER (ORDER BY perc_non_white) AS race_ntile
        , PERCENT_RANK() OVER (ORDER BY pop_density DESC) AS density_ntile
        , PERCENT_RANK() OVER (ORDER BY dist_to_transit) AS transit_dist_ntile
        , PERCENT_RANK() OVER (ORDER BY transit_freq DESC) AS transit_freq_ntile
        FROM cc_transpo_needs
        WHERE 1=1
    )
    UPDATE cc_transpo_needs AS l
    SET hh_inc_ntile = p.hh_inc_ntile
    , age_ntile = p.age_ntile
    , race_ntile = p.race_ntile
    , density_ntile = p.density_ntile
    , transit_dist_ntile = p.transit_dist_ntile
    , transit_freq_ntile = p.transit_freq_ntile
    FROM percentiles p 
    WHERE 1=1
    AND p.geoid10 = l.geoid10
    ;
"""

engine.execute(query)

In [6]:
# Index

query = """
    ALTER TABLE cc_transpo_needs ADD COLUMN transpo_needs_score FLOAT;
    UPDATE cc_transpo_needs 
    SET transpo_needs_score = (hh_inc_ntile + age_ntile + race_ntile + density_ntile + 
                                transit_dist_ntile + transit_freq_ntile)/6
    ;
"""

engine.execute(query)